In [55]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
car=pd.read_csv('mtcars.csv')
car.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [58]:
x=car.drop(['model','am'],axis=1)
x.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,3,2


In [59]:
y=car[['am']]

In [60]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [61]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train[['mpg','disp','hp','drat','wt','qsec','carb']]=sc.fit_transform(x_train[['mpg','disp','hp','drat','wt','qsec','carb']])
x_train_scaled=x_train

In [62]:
x_train_scaled.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,gear,carb
4,-0.207042,8,1.039383,0.508837,-0.761861,0.253449,-0.541833,0,3,-0.431788
16,-0.876883,8,1.705848,1.393452,-0.621895,2.216132,-0.278691,0,3,1.295363
5,-0.307518,6,-0.085277,-0.617038,-1.444196,0.274054,1.563303,1,3,-1.295363
13,-0.793153,8,0.337928,0.589256,-0.901828,0.603744,0.102865,0,3,0.431788
11,-0.592200,8,0.337928,0.589256,-0.901828,0.902525,-0.291848,0,3,0.431788


In [63]:
from sklearn.linear_model import LogisticRegression
lgr=LogisticRegression()

In [64]:
from sklearn.feature_selection import RFE
rfe=RFE(lgr,6)

In [65]:
rfe=rfe.fit(x_train_scaled,y_train)

In [66]:
list(zip(x_train_scaled,rfe.support_,rfe.ranking_))

[('mpg', True, 1),
 ('cyl', True, 1),
 ('disp', False, 2),
 ('hp', False, 5),
 ('drat', True, 1),
 ('wt', True, 1),
 ('qsec', True, 1),
 ('vs', False, 3),
 ('gear', True, 1),
 ('carb', False, 4)]

In [67]:
sel_col=x_train_scaled.columns[rfe.support_]

In [68]:
x_train_scaled_rfe=x_train_scaled[sel_col]
x_train_scaled_rfe.head()

,mpg,cyl,drat,wt,qsec,gear
4,-0.207042,8,-0.761861,0.253449,-0.541833,3
16,-0.876883,8,-0.621895,2.216132,-0.278691,3
5,-0.307518,6,-1.444196,0.274054,1.563303,3
13,-0.793153,8,-0.901828,0.603744,0.102865,3
11,-0.592200,8,-0.901828,0.902525,-0.291848,3


In [69]:
lgr.fit(x_train_scaled_rfe,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
y_pred_lgr_train=lgr.predict(x_train_scaled_rfe)

In [71]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
print('Accuresy: ',accuracy_score(y_train,y_pred_lgr_train))

Accuresy:  1.0


In [72]:
print("Confussion Matrix:\n\n",confusion_matrix(y_train,y_pred_lgr_train))

Confussion Matrix:

 [[14  0]
 [ 0  8]]


In [75]:
x_test[['mpg','disp','hp','drat','wt','qsec','carb']]=sc.transform(x_test[['mpg','disp','hp','drat','wt','qsec','carb']])

In [76]:
x_test_rfe=x_test[x_train_scaled_rfe.columns]

In [77]:
y_pred_lgr_test=lgr.predict(x_test_rfe)

In [78]:
print('Accuresy: ',accuracy_score(y_test,y_pred_lgr_test))

Accuresy:  1.0


In [79]:
print("Confussion Matrix:\n\n",confusion_matrix(y_test,y_pred_lgr_test))

Confussion Matrix:

 [[5 0]
 [0 5]]


In [83]:
print('Classification result:\n',classification_report(y_test,y_pred_lgr_test))

Classification result:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [15]:
import statsmodels.api as sm

In [16]:
x_train_scaled_rfe_sm=sm.add_constant(x_train_scaled_rfe)

In [17]:
x_train_scaled_rfe_sm.head()

,const,mpg,cyl,drat,wt,qsec,gear
4,1.0,-0.207042,8,-0.761861,0.253449,-0.541833,3
16,1.0,-0.876883,8,-0.621895,2.216132,-0.278691,3
5,1.0,-0.307518,6,-1.444196,0.274054,1.563303,3
13,1.0,-0.793153,8,-0.901828,0.603744,0.102865,3
11,1.0,-0.592200,8,-0.901828,0.902525,-0.291848,3


In [20]:
y_train.head()

,am
4,0
16,0
5,0
13,0
11,0


In [ ]:
y_train

In [80]:
logm=sm.GLM(y_train,x_train_scaled_rfe_sm,family=sm.families.Binomial()).fit()

PerfectSeparationError: Perfect separation detected, results not available

In [ ]:
logm.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['Features']=x_train_scaled_rfe_sm.columns
vif['VIF']=[variance_inflation_factor(x_train_scaled_rfe_sm.values,i) for i in range(x_train_scaled_rfe_sm.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train_scaled_rfe_sm=x_train_scaled_rfe_sm.drop('cyl',axis=1,)

In [ ]:
logm1=sm.GLM(y_train,x_train_scaled_rfe_sm,family=sm.families.Binomial()).fit()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train_scaled_rfe_sm.columns
vif['VIF']=[variance_inflation_factor(x_train_scaled_rfe_sm.values,i) for i in range(x_train_scaled_rfe_sm.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [ ]:
y_train=y_train

In [ ]:
y_train_pred_sm=logm1.predict(x_train_scaled_rfe_sm)

In [ ]:
print('Accuracy Score:\n')
print(accuracy_score(y_train,y_train_pred_sm))
print('confution')